In [30]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [31]:
import xgboost
from xgboost import XGBClassifier
xgboost.__version__

'1.7.4'

In [32]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

G3Depth_TPM_merged_lats = pd.read_csv('../../g3Depth/g3Depth_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

,PF00001,PF00002,PF00003,PF00004,PF00005,PF00006,PF00007,PF00008,PF00009,PF00010,...,PF20009,PF20030,PF20043,PF20067,PF20071,PF20102,PF20133,PF20143,PF20144,PF20145
0,0.000000,0.000000,758.695203,1832.372772,2627.410068,12.693236,0.0,2.341709,756.539101,20.434529,...,0.0,21.960879,0.000000,0.0,0.000000,0.000000,44.508799,193.271660,0.0,17.575702
1,112.717514,0.000000,0.000000,2647.360925,1915.120991,52.316307,0.0,0.000000,221.116922,87.151476,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,6.250765,0.0,0.000000
2,1.468958,0.000000,0.000000,1080.024247,3128.618737,1097.395938,0.0,0.000195,900.009082,0.000000,...,0.0,1.557527,3.985164,0.0,0.000000,6.827003,3.243305,19.922488,0.0,0.000000
3,3.627817,0.000000,7.707109,2928.668085,1923.927420,1588.542407,0.0,60.327535,6783.173110,0.000000,...,0.0,12.989171,0.000000,0.0,17.441676,3.440638,1.248229,8.734192,0.0,4.836338
4,0.000000,0.000000,0.000000,134.836688,555.907656,0.000000,0.0,0.730475,26.559945,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,22.266101,2.712314,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,0.000000,0.000000,2064.681237,1316.346448,1015.085808,41.390166,0.0,0.000000,538.553199,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,40.634365,1406.493072,0.000000,0.0,59.546949
310,0.000000,0.000000,20.069611,1161.167903,923.906976,462.079006,0.0,512.979750,1721.218866,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,84.403508,0.000000,0.0,233.528413
311,0.000000,0.000000,405.641341,1482.520127,1707.400394,1494.753537,0.0,0.000000,7139.490523,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,83.382649,0.0,0.000000
312,0.000000,15.326686,4.640790,2761.886369,1894.016307,2620.602456,0.0,20.967695,3659.257951,8.806177,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,559.395450,0.000000,0.0,20.214729


In [33]:
G3Depth_TPM_merged_lats.columns[(G3Depth_TPM_merged_lats == 0).all()]

Index(['PF14259'], dtype='object')

In [35]:
features = features['pfam']

In [42]:
len(features)

183

In [43]:
G3Depth_TPM_merged_lats = G3Depth_TPM_merged_lats[features]

In [45]:
G3Depth_TPM_merged_lats.columns[(G3Depth_TPM_merged_lats == 0).all()]

Index(['PF14259'], dtype='object')

In [46]:
train_data = train_data[features]

In [47]:
le = LabelEncoder()

In [48]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [49]:
G3Depth = scaler.transform(G3Depth_TPM_merged_lats)

In [50]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.5, learning_rate=0.2, max_depth=3, n_estimators=100, reg_lambda=0.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [51]:
xg_predictions_G3Depth = model.predict(G3Depth)

In [53]:
prob = model.predict_proba(G3Depth)

In [54]:
G3Depth_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G3Depth})

In [55]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [57]:
G3Depth_predictions.to_csv('../../g3Depth/g3Depth_trophicModePredictions_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)

In [58]:
probDF.to_csv('../../g3Depth/g3Depth_trophicModePredictionsProbabilities_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)